# Data Exploration Wouter

In [2]:
# import packages
import os 
import pickle

import re
from datetime import datetime

import requests

import pytz

import pandas as pd
import numpy as np

import ast


import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import array_contains


import matplotlib.pyplot as plt
import seaborn as sns

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [3]:
# import findspark
import findspark

# initialize findspark with spark directory

#ALWAYS HAVE TO BE CHANGED 
findspark.init("/Users/wouterdewitte/spark/")

# import pyspark
import pyspark
# create spark context
sc = pyspark.SparkContext()
# create spark session 
spark = pyspark.sql.SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/28 08:32:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [4]:
#set this path to your path, for some reason I have an error 
#reading in all the files
import os
path_json = ".././../data/Topic_vegan/*.json"

In [5]:
df_json = spark.read.json(path_json)

22/11/28 08:33:47 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
# select interesting features
import pyspark.sql.functions as F

df = df_json.select(F.col("user.name"),
                    F.col("user.screen_name"),
                    F.col("created_at"), 
                    F.col("full_text"),
                    F.col("user.followers_count"),
                    F.col("retweet_count"),
                    F.col("favorite_count"),
                    F.col("entities.hashtags"))
df.limit(10).toPandas()

,name,screen_name,created_at,full_text,followers_count,retweet_count,favorite_count,hashtags
0,のり/Nori,nori_k_629,Mon Apr 04 10:09:55 +0000 2022,RT @ohmpawatt: เพื่อนๆคิดถึงผมมั้ยยย ถ้าคิดถึง...,139,3582,0,[]
1,alice,myn4meizalize,Mon Apr 04 10:09:54 +0000 2022,RT @mynameisnanon: คิดถึงกันป่าว ถ้าคิดถึงต้อง...,655,3837,0,[]
2,Karen Reed 🌸,kandk670,Mon Apr 04 10:09:54 +0000 2022,@trudiebakescake Organic coconut oil in a jar ...,711,0,0,[]
3,ハル):),patlnwza55,Mon Apr 04 10:09:52 +0000 2022,RT @ohmpawatt: เพื่อนๆคิดถึงผมมั้ยยย ถ้าคิดถึง...,236,3582,0,[]
4,alice,myn4meizalize,Mon Apr 04 10:09:52 +0000 2022,RT @ohmpawatt: เพื่อนๆคิดถึงผมมั้ยยย ถ้าคิดถึง...,655,3582,0,[]
5,ﾌｧﾙﾄﾞﾗっ子organic有機💙💻,organic_yusai,Mon Apr 04 10:09:52 +0000 2022,マジでピンチ助けて自転車ガガガガ,291,0,1,[]
6,のり/Nori,nori_k_629,Mon Apr 04 10:09:50 +0000 2022,RT @mynameisnanon: คิดถึงกันป่าว ถ้าคิดถึงต้อง...,139,3837,0,[]
7,｡◕‿◕｡𝑱𝒆 𝒕'𝒂𝒊𝒎𝒆 🐶🧡✨,MyFnlovely97,Mon Apr 04 10:09:50 +0000 2022,RT @ohmpawatt: เพื่อนๆคิดถึงผมมั้ยยย ถ้าคิดถึง...,245,3582,0,[]
8,Sang™,asan_gk,Mon Apr 04 10:09:50 +0000 2022,RT @NotechAna: Am I the only one who types in ...,2065,374,0,[]
9,Trysia ):)▪︎never let me go▪︎,Winnie_thephuu,Mon Apr 04 10:09:48 +0000 2022,RT @ohmpawatt: เพื่อนๆคิดถึงผมมั้ยยย ถ้าคิดถึง...,379,3582,0,[]


In [7]:
# https://developer.twitter.com/en/docs/twitter-ads-api/timezones
# function to convert Twitter date string format
def getDate(date):
    if date is not None:
        return str(datetime.strptime(date,'%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo=pytz.UTC).strftime("%Y-%m-%d %H:%M:%S"))
    else:
        return None

# UDF declaration
date_udf = F.udf(getDate, StringType())

# apply udf
df = df.withColumn('post_created_at', F.to_utc_timestamp(date_udf("created_at"), "UTC"))

In [8]:
#drop duplicates and retweets 
df = df.filter(~F.col("full_text").startswith("RT"))\
                        .drop_duplicates().cache()
#sorting such when dropping later we only keep the most recent post 
df = df.sort("post_created_at", ascending=False)
#removing spam accounts 
df = df.drop_duplicates(["full_text", "screen_name"])
                        
df.printSchema()
#df.count() #1340938 

root
 |-- name: string (nullable = true)
 |-- screen_name: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- full_text: string (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- indices: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- text: string (nullable = true)
 |-- post_created_at: timestamp (nullable = true)



In [9]:
# get first post
first_post = F.min('post_created_at').alias('earliest')
# get latest post
latest_post = F.max('post_created_at').alias('latest')
# show tweet period in our dataset
df.select(first_post, latest_post).show()

+-------------------+-------------------+
|           earliest|             latest|
+-------------------+-------------------+
|2021-10-12 16:08:51|2022-10-11 23:17:33|
+-------------------+-------------------+



# The evolution of tweet activity

Look at the frequency of tweets per month.

In [9]:
def tweet_activity_monthly(keyword, df):
    
    df = df.filter(df.hashtags.contains(keyword))

    # freq_month
    freq_month = df.withColumn("year", year(df["post_created_at"]))
    freq_month = freq_month.withColumn("month", month(df["post_created_at"]))

    freq_month = freq_month.groupBy('year', 'month').agg(countDistinct("full_text"))\
                    .withColumnRenamed("count(full_text)", "freq") \
                        .sort('year', 'month', ascending = True)
    freq_month = freq_month.select(concat_ws('_',freq_month.year, freq_month.month)\
                            .alias('date'), 'freq').toPandas()

    fig = px.bar(freq_month, x='date', y='freq')

    # Add figure title
    fig.update_layout(
        title_text="Tweet Activity Monthly " + keyword,
        title_x = 0.5
    )

    # add axes
    fig.update_xaxes(title_text="<b>Month</b>")
    fig.update_yaxes(title_text="<b>Amount of tweets</b>")

    fig.show()

In [43]:
tweet_activity_monthly("healthylifestyle", df)

Look at the frequency of tweets per week.

In [44]:
def tweet_activity_weekly(keyword, df):
    df = df.filter(df.full_text.contains(keyword))

    freq_week = df.withColumn("year", year(df["post_created_at"]))
    freq_week = freq_week.withColumn('week', weekofyear('post_created_at'))

    freq_week = freq_week.groupBy('year', 'week').agg(countDistinct("full_text"))\
                    .withColumnRenamed("count(full_text)", "freq") \
                        .sort('year', 'week', ascending = True)
    freq_week = freq_week.select(concat_ws('_',freq_week.year, freq_week.week)\
                            .alias('date'), 'freq').toPandas()

    fig = px.line(freq_week, x='date', y='freq')

    # Add figure title
    fig.update_layout(
        title_text="Tweet Activity Weekly",
        title_x = 0.5
    )

    # add axes
    fig.update_xaxes(title_text="<b>Week</b>")
    fig.update_yaxes(title_text="<b>Amount of tweets</b>")

    fig.show()


In [45]:
tweet_activity_weekly("vegan", df)

# The evolution of tweet engagement 

In [13]:
def eng_weekly(keyword, df):
    df = df.filter(df.full_text.contains(keyword))

    eng_weekly = df.withColumn('year', year(df["post_created_at"]))
    eng_weekly = eng_weekly.withColumn('week', weekofyear('post_created_at'))

    eng_weekly = eng_weekly.groupBy("year", "week") \
                        .agg(sum("retweet_count").alias("retweets"), sum("favorite_count").alias("favorites")) \
                        .sort("year", "week", ascending = True)

    eng_weekly = eng_weekly.select(concat_ws('_',eng_weekly.year, eng_weekly.week)\
                            .alias('week'), 'retweets', 'favorites').toPandas()

    fig = px.line(eng_weekly, x='week', y=eng_weekly.columns[1:3])

    # Add figure title
    fig.update_layout(
        title_text="Retweets vs favorites",
        title_x = 0.5
    )

    # add axes
    fig.update_xaxes(title_text="<b>Week</b>")
    fig.update_yaxes(title_text="<b>Amount</b>")

    fig.show()


In [14]:
eng_weekly("vegan", df)

# The level of social media activity vs level of engagement

Engagement on Twitter is measured by the number of retweets, follows, replies, favorites, and other people’s reactions to your tweets, including the clicks on the links and hashtags in those tweets. Your Twitter engagement rate is your engagement figure divided by the number of impressions on the tweet.

According to our 2022 Social Media Industry Benchmark Report, the overall median engagement rate on Twitter is 0.037%. This is the rate for brands across all industries, from fashion to nonprofits. On average, these brands are also posting about 5 times per week.

https://www.rivaliq.com/blog/good-engagement-rate-twitter/

In [50]:
def eng_rate(keyword, df):
    df = df.filter(df.full_text.contains(keyword))

    eng_rate = df.withColumn('eng_rate', ((df['favorite_count'] + df['retweet_count'])/df['followers_count']))

    eng_rate = eng_rate.withColumn('year', year(df["post_created_at"]))
    eng_rate = eng_rate.withColumn('week', weekofyear('post_created_at'))

    eng_rate_weekly = eng_rate.groupBy("year", "week") \
                        .agg(avg("eng_rate").alias("eng_rate")) \
                        .sort("year","week", ascending = True)
    eng_rate_weekly = eng_rate_weekly.select(concat_ws('_',eng_rate_weekly.year, eng_rate_weekly.week)\
                            .alias('week'), 'eng_rate').toPandas()

    freq_week = df.withColumn("year", year(df["post_created_at"]))
    freq_week = freq_week.withColumn('week', weekofyear('post_created_at'))

    freq_week = freq_week.groupBy('year', 'week').agg(countDistinct("full_text"))\
                    .withColumnRenamed("count(full_text)", "freq") \
                        .sort('year', 'week', ascending = True)
    freq_week = freq_week.select(concat_ws('_',freq_week.year, freq_week.week)\
                            .alias('week'), 'freq').toPandas()

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x = freq_week["week"],y = freq_week["freq"], name="Amount of tweets"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x = eng_rate_weekly["week"], y = eng_rate_weekly["eng_rate"], name="Engagement rate"),
        secondary_y=True,
    )

    # Add figure title
    fig.update_layout(
        title_text="Amount of tweets compared to engagement rate",
        title_x = 0.5,
        autosize = True
    )


    # Set x-axis title
    fig.update_xaxes(title_text="<b>week</b>")

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Amount</b> of tweets", secondary_y=False)
    fig.update_yaxes(title_text="<b>Engagement</b> rate", secondary_y=True)

    fig.show()
    

In [66]:
eng_rate("vegan", df)

# The volume of influencer activity vs level of engagement 

We define an influencer as an account with the following characteristics:

- a lot of followers => follower_count > 10000
- there is a high engagement rate on their tweets which shows their influence => er > 0.05
- tweet frequency is high enough => freq_weekly > 20

In [8]:
def get_influencers(follower_count_tresh, eng_rate_tresh, freq_week_tresh):

    # get all users with their amount of followers
    influencers = df.groupBy("screen_name") \
                    .agg(first("followers_count").alias("followers_count"))

    # average engagement rate for each user
    eng_rate = df.withColumn('eng_rate', ((df['favorite_count'] + df['retweet_count'])/df['followers_count']))

    eng_rate_user = eng_rate.groupBy("screen_name") \
                            .agg(avg("eng_rate").alias("eng_rate"))

    # average freq_weekly per user
    freq_week = df.withColumn("year", year(df["post_created_at"]))
    freq_week = freq_week.withColumn('week', weekofyear('post_created_at'))

    freq_week = freq_week.groupBy('screen_name', 'year', 'week').agg(countDistinct("full_text"))\
                    .withColumnRenamed("count(full_text)", "freq") \
                        .sort('screen_name', 'year', 'week', ascending = True)
    freq_week = freq_week.select('screen_name', 'freq')

    freq_week = freq_week.groupby("screen_name").agg(avg(freq_week.freq).alias('freq'))

    # put the data together
    data_joined = eng_rate_user.join(influencers, "screen_name").join(freq_week, "screen_name")

    # filter the data
    data_joined = data_joined.filter((data_joined.followers_count > follower_count_tresh) & (data_joined.eng_rate > eng_rate_tresh) & (data_joined.freq > freq_week_tresh))
    
    # show the data
    data_joined.show()
    return data_joined

In [58]:
influencers = get_influencers(10000, 0.04, 2)

22/11/25 17:40:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:40:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:40:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:40:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:40:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:40:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:40:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:40:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:40:21 WARN RowBasedKeyValueBatch: Calling spill() on

22/11/25 17:40:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:40:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:40:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:40:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:40:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-------------+-------------------+---------------+----+
|  screen_name|           eng_rate|followers_count|freq|
+-------------+-------------------+---------------+----+
|  ateenyalien|0.09717012618187412|          32730| 2.5|
|MissAudreyBoo|0.10823612246832817|          13885| 4.0|
|       itsTFC|0.04170045922053734|          12135| 7.0|
|  teonawrites|0.16173973342805512|          29310| 4.0|
+-------------+-------------------+---------------+----+



In [59]:
df = df.alias('df')
influencers = influencers.alias('influencers')
influencers_tweets = df.join(influencers, "screen_name").select("df.*")
influencers_tweets.show()

In [60]:
eng_rate("vegan", influencers_tweets)

22/11/25 17:41:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:41:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:41:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:41:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:41:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:41:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:41:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:41:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:41:47 WARN RowBasedKeyValueBatch: Calling spill() on

22/11/25 17:41:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:41:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:41:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:41:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:41:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/11/25 17:43:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:43:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:43:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:43:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:43:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:43:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:43:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:43:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:43:15 WARN RowBasedKeyValueBatch: Calling spill() on

22/11/25 17:43:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:43:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:43:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:43:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/11/25 17:43:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


# Dependent variable: Google Trends

In [29]:
from pytrendsasync.request import TrendReq

from pytrends import dailydata
pytrends = TrendReq(hl='en-US', tz=360, timeout=10, proxies=['https://34.203.233.13:80',])


In [30]:
def get_dependent(keyword):
    # import again to avoid Google error
    from pytrendsasync.request import TrendReq
    from pytrends import dailydata

    # get the data
    df = dailydata.get_daily_data(keyword, 2021, 10, 2022, 10)

    # only get relevant keyword column
    df = df[keyword].reset_index()

    # create the binary dependent variable
    df[keyword + "_yest"] = df[keyword].shift(1)
    df["dependent_" + keyword] = df.apply(lambda x:  1 if x[keyword] > x[keyword + "_yest"] else 0, axis=1) # 1 if popularity rose compared to yesterday, otherwise 0
    
    df = df[["date", "dependent_" + keyword]]
    df = df.iloc[3: , :] # drop first 2 rows because insufficient data
    
    return df


In [33]:
df = get_dependent("vegan")

vegan:2021-10-01 2021-10-31
vegan:2021-11-01 2021-11-30
vegan:2021-12-01 2021-12-31
vegan:2022-01-01 2022-01-31
vegan:2022-02-01 2022-02-28
vegan:2022-03-01 2022-03-31
vegan:2022-04-01 2022-04-30
vegan:2022-05-01 2022-05-31
vegan:2022-06-01 2022-06-30
vegan:2022-07-01 2022-07-31
vegan:2022-08-01 2022-08-31
vegan:2022-09-01 2022-09-30
vegan:2022-10-01 2022-10-31


In [36]:
df["dependent_vegan"].value_counts()

1    207
0    186
Name: dependent_vegan, dtype: int64